## Horse racing prediction  

This is an experiment to predict the outcome of horse racing based on past 5 race results, jockey, and trainer.



## Prepare data

In [1]:
# import pymongo
import numpy as np
from pymongo import MongoClient
client = MongoClient()

db = client.keiba
training_data = db.training_data_Kisyu_Kyusya_1_race_5_with_odds
data_models = db.data_models_Kisyu_Kyusya_1_race_5_with_odds

In [2]:
#
# > db.training_data_Kisyu_Kyusya_1_race_5_with_odds.count({'input_x_count':105})
# 9245046
#
# > db.training_data_Kisyu_Kyusya_1_race_5_with_odds.count()
# 9247112
#
# for some reasons, 2,066 record don't have 105 features...
# 

training_data_cursor = training_data.find({'random_index': {'$gt':19}}, no_cursor_timeout=True)
validation_data_cursor = training_data.find({'random_index': {'$lt':20}}, no_cursor_timeout=True)

# too slow to use
# training_data_cursor = training_data.find({'random_index': {'$gt':19}, 'input_x_count':105})
# validation_data_cursor = training_data.find({'random_index': {'$lt':20, 'input_x_count':105}})

In [3]:
training_data_count = training_data_cursor.count()
print("training_data_count: {}".format(training_data_count))

validation_data_count = validation_data_cursor.count()
print("validation_data_count: {}".format(validation_data_count))

training_data_count: 7488116
validation_data_count: 1756930


In [4]:
# get std and mean. we use data_model later
mean_and_std = data_models.find_one({})

In [5]:
# get batch as generator
# batch_size is mini batch size
# data_type is training or validation

#
# pymongo.errors.CursorNotFound: cursor id '…' not valid at server
#

# do not use try exception model,
# can not chatch error if we use it.


def data_generator(batch_size, data_type):
    
#     still_have_data_flg = True
    
    input_X = np.zeros(shape=(batch_size, 105), dtype=float)
    target_Y = np.zeros(shape=(batch_size, 1), dtype=float)
    
    while True:
        
        for idx1 in range(batch_size):
            
            # get one row
            data1 = None
            if data_type == 'validation':
                data1 = validation_data_cursor.next()
            else:
                data1 = training_data_cursor.next()

            # normalize x values
            for idx2 in data1['input_x_object']:

                # get model data which contains mean and std
                x1 = data1['input_x_object'][idx2]

                mean_name = 'input_x_avg_'+idx2
                mean_value = mean_and_std['mean_and_std'][mean_name]

                std_name = 'input_x_std_'+idx2
                std_value = mean_and_std['mean_and_std'][std_name]

                normarized_x = (x1 - mean_value) / std_value
                input_X[idx1, int(idx2)] = normarized_x

            # normarize y value
            y1 = data1['target_y']
            y_mean_value = mean_and_std['mean_and_std']['target_y_mean']
            y_std_value = mean_and_std['mean_and_std']['target_y_stddev']
            normalized_y = (y1 - y_mean_value) / y_std_value

            target_Y[idx1] = normalized_y

            yield (input_X, target_Y)


In [6]:
#  y is 1 or 0

def data_generator_binary(batch_size, data_type):
    
#     still_have_data_flg = True
    
    input_X = np.zeros(shape=(batch_size, 105), dtype=float)
    target_Y = np.zeros(shape=(batch_size, 1), dtype=float)
    
    while True:
        
        for idx1 in range(batch_size):
            
            # get one row
            data1 = None
            if data_type == 'validation':
                data1 = validation_data_cursor.next()
            else:
                data1 = training_data_cursor.next()

            # normalize x values
            for idx2 in data1['input_x_object']:

                # get model data which contains mean and std
                x1 = data1['input_x_object'][idx2]

                mean_name = 'input_x_avg_'+idx2
                mean_value = mean_and_std['mean_and_std'][mean_name]

                std_name = 'input_x_std_'+idx2
                std_value = mean_and_std['mean_and_std'][std_name]

                normarized_x = (x1 - mean_value) / std_value
                input_X[idx1, int(idx2)] = normarized_x

            # normarize y value
            y1 = data1['target_y']
            if y1 >= 0:
                target_Y[idx1] = 1
            else:
                target_Y[idx1] = 0
#             y_mean_value = mean_and_std['mean_and_std']['target_y_mean']
#             y_std_value = mean_and_std['mean_and_std']['target_y_stddev']
#             normalized_y = (y1 - y_mean_value) / y_std_value

#             target_Y[idx1] = normalized_y

            yield (input_X, target_Y)


## Create model 

In [7]:
# import dependancies
import keras
from keras import metrics, initializers

from keras_tqdm import TQDMNotebookCallback
from keras.layers import Dropout, Dense, LeakyReLU, BatchNormalization, Activation
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam, RMSprop


Using TensorFlow backend.
/home/kouohhashi/anaconda3/envs/aind-vui/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [8]:
# model 1: 3 layers, LeakyReLU, and dropout
model_1 = Sequential()

model_1.add(Dense(128, input_shape=(105,), activation=None))
model_1.add(LeakyReLU(alpha=0.3))
model_1.add(Dropout(0.2))

model_1.add(Dense(256, activation=None))
model_1.add(LeakyReLU(alpha=0.3))
model_1.add(Dropout(0.2))

model_1.add(Dense(128, activation=None))
model_1.add(LeakyReLU(alpha=0.3))
model_1.add(Dropout(0.2))

model_1.add(Dense(1, activation=None))

model_1.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=[metrics.mae])

model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               13568     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
__________

In [9]:
# training model_1

# add checkpointer
save_model_name = "keiba_model_1.h5"
checkpointer = ModelCheckpoint(filepath='results/'+save_model_name, verbose=0)

minibatch_size = 32

steps_per_epoch = training_data_count // minibatch_size
validation_steps = validation_data_count // minibatch_size

model_1.fit_generator(generator=data_generator(batch_size=minibatch_size, data_type='training'),
                    steps_per_epoch=steps_per_epoch,
                    validation_data=data_generator(batch_size=minibatch_size, data_type='validation'),
                    validation_steps=validation_steps,
                    epochs=20,
                    callbacks=[checkpointer])

Epoch 1/20
234003/234003 [==============================] - 974s 4ms/step - loss: 0.2908 - mean_absolute_error: 0.2090 - val_loss: 1.3641 - val_mean_absolute_error: 0.4020
Epoch 2/20
234003/234003 [==============================] - 965s 4ms/step - loss: 0.3030 - mean_absolute_error: 0.2161 - val_loss: 0.9627 - val_mean_absolute_error: 0.2172
Epoch 3/20
234003/234003 [==============================] - 963s 4ms/step - loss: 0.2889 - mean_absolute_error: 0.2142 - val_loss: 1.0180 - val_mean_absolute_error: 0.2672
Epoch 4/20
234003/234003 [==============================] - 963s 4ms/step - loss: 0.3061 - mean_absolute_error: 0.2230 - val_loss: 1.0920 - val_mean_absolute_error: 0.3859
Epoch 5/20
234003/234003 [==============================] - 968s 4ms/step - loss: 0.3030 - mean_absolute_error: 0.2225 - val_loss: 1.4493 - val_mean_absolute_error: 0.5858
Epoch 6/20
234003/234003 [==============================] - 965s 4ms/step - loss: 0.3015 - mean_absolute_error: 0.2251 - val_loss: 1.0960 - 

## Model 2  


In [14]:
# model 2: simple one with 2 layers, LeakyReLU and dropout
model_2 = Sequential()

# keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None)
# kernel_initializer=initializers.RandomNormal(stddev=0.001)
model_2.add(Dense(64,
                  input_shape=(105,), 
                  activation=None, 
                  bias_initializer=initializers.RandomNormal(stddev=0.001),
                  kernel_initializer=initializers.RandomNormal(stddev=0.001)))
model_2.add(LeakyReLU(alpha=0.2))
model_2.add(Dropout(0.2))

model_2.add(Dense(32, activation=None))
model_2.add(LeakyReLU(alpha=0.2))
model_2.add(Dropout(0.2))

model_2.add(Dense(1, activation=None))

keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)

# SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
# Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model_2.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='mean_squared_error',
              metrics=[metrics.mae])

model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                6784      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total para

In [15]:
# training model_2

# add checkpointer
save_model_name = "keiba_model_2.h5"
checkpointer = ModelCheckpoint(filepath='results/'+save_model_name, verbose=0)

minibatch_size = 32

steps_per_epoch = training_data_count // minibatch_size
validation_steps = validation_data_count // minibatch_size

model_2.fit_generator(generator=data_generator(batch_size=minibatch_size, data_type='training'),
                    steps_per_epoch=steps_per_epoch,
                    validation_data=data_generator(batch_size=minibatch_size, data_type='validation'),
                    validation_steps=validation_steps,
                    epochs=10,
                    callbacks=[checkpointer])

Epoch 1/10
234003/234003 [==============================] - 561s 2ms/step - loss: 0.9403 - mean_absolute_error: 0.3288 - val_loss: 0.9947 - val_mean_absolute_error: 0.1900
Epoch 2/10
234003/234003 [==============================] - 559s 2ms/step - loss: 1.1812 - mean_absolute_error: 0.3479 - val_loss: 1.3071 - val_mean_absolute_error: 0.4377
Epoch 3/10
234003/234003 [==============================] - 562s 2ms/step - loss: 1.1814 - mean_absolute_error: 0.3510 - val_loss: 0.9916 - val_mean_absolute_error: 0.2160
Epoch 4/10
234003/234003 [==============================] - 561s 2ms/step - loss: 1.2087 - mean_absolute_error: 0.3656 - val_loss: 0.9631 - val_mean_absolute_error: 0.2103
Epoch 5/10
234003/234003 [==============================] - 559s 2ms/step - loss: 1.0372 - mean_absolute_error: 0.3472 - val_loss: 1.2585 - val_mean_absolute_error: 0.3393
Epoch 6/10
234003/234003 [==============================] - 557s 2ms/step - loss: 1.2197 - mean_absolute_error: 0.3497 - val_loss: 0.9334 - 

## model 3

In [7]:
# model 3: simple one  + batch normalization
model_3 = Sequential()


# keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None)
model_3.add(Dense(64,
                  input_shape=(105,),
                  activation=None,
                  kernel_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                  bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None)))
model_3.add(BatchNormalization())
model_3.add(LeakyReLU(alpha=0.2))
model_3.add(Dropout(0.2))

model_3.add(Dense(32, activation=None))
model_3.add(BatchNormalization())
model_3.add(LeakyReLU(alpha=0.2))
model_3.add(Dropout(0.2))

model_3.add(Dense(1, activation=None))

model_3.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='mean_squared_error',
              metrics=[metrics.mae])

model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                6784      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32)                0         
__________

In [8]:
# training model_3

# add checkpointer
save_model_name = "keiba_model_3.h5"
checkpointer = ModelCheckpoint(filepath='results/'+save_model_name, verbose=0)

minibatch_size = 64

steps_per_epoch = training_data_count // minibatch_size
validation_steps = validation_data_count // minibatch_size

model_3.fit_generator(generator=data_generator(batch_size=minibatch_size, data_type='training'),
                    steps_per_epoch=steps_per_epoch,
                    validation_data=data_generator(batch_size=minibatch_size, data_type='validation'),
                    validation_steps=validation_steps,
                    epochs=10,
                    callbacks=[checkpointer])

Epoch 1/10
117001/117001 [==============================] - 418s 4ms/step - loss: 0.7740 - mean_absolute_error: 0.2722 - val_loss: 1.5539 - val_mean_absolute_error: 0.5312
Epoch 2/10
117001/117001 [==============================] - 405s 3ms/step - loss: 0.8763 - mean_absolute_error: 0.2767 - val_loss: 1.0532 - val_mean_absolute_error: 0.2857
Epoch 3/10
117001/117001 [==============================] - 402s 3ms/step - loss: 0.8814 - mean_absolute_error: 0.2769 - val_loss: 1.0289 - val_mean_absolute_error: 0.3160
Epoch 4/10
117001/117001 [==============================] - 408s 3ms/step - loss: 0.9084 - mean_absolute_error: 0.2758 - val_loss: 1.2050 - val_mean_absolute_error: 0.3504
Epoch 5/10
117001/117001 [==============================] - 408s 3ms/step - loss: 0.8514 - mean_absolute_error: 0.2671 - val_loss: 1.2065 - val_mean_absolute_error: 0.3319
Epoch 6/10
117001/117001 [==============================] - 405s 3ms/step - loss: 0.9047 - mean_absolute_error: 0.2732 - val_loss: 0.9973 - 

## Model 4

In [9]:
# model 4
model_4 = Sequential()

model_4.add(Dense(64,
                  input_shape=(105,),
                  activation=None,
                  kernel_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                  bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None)))
# model_4.add(BatchNormalization())
model_4.add(LeakyReLU(alpha=0.2))
# model_4.add(Dropout(0.2))

model_4.add(Dense(256, activation=None))
# model_4.add(BatchNormalization())
model_4.add(LeakyReLU(alpha=0.2))
# model_4.add(Dropout(0.2))

model_4.add(Dense(128, activation=None))
# model_4.add(BatchNormalization())
model_4.add(LeakyReLU(alpha=0.2))
# model_4.add(Dropout(0.2))

model_4.add(Dense(1, activation=None))

# SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)?
# model_4.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
model_4.compile(optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True),
              loss='mean_squared_error',
              metrics=[metrics.mae])

model_4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                6784      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total para

In [10]:
# training model_4

# add checkpointer
save_model_name = "keiba_model_4.h5"
checkpointer = ModelCheckpoint(filepath='results/'+save_model_name, verbose=0)

minibatch_size = 64

steps_per_epoch = training_data_count // minibatch_size
validation_steps = validation_data_count // minibatch_size

model_4.fit_generator(generator=data_generator(batch_size=minibatch_size, data_type='training'),
                    steps_per_epoch=steps_per_epoch,
                    validation_data=data_generator(batch_size=minibatch_size, data_type='validation'),
                    validation_steps=validation_steps,
                    epochs=10,
                    callbacks=[checkpointer])

Epoch 1/10
117001/117001 [==============================] - 461s 4ms/step - loss: 0.4133 - mean_absolute_error: 0.2107 - val_loss: 1.2420 - val_mean_absolute_error: 0.3082
Epoch 2/10
117001/117001 [==============================] - 456s 4ms/step - loss: 0.3529 - mean_absolute_error: 0.1918 - val_loss: 1.2498 - val_mean_absolute_error: 0.3263
Epoch 3/10
117001/117001 [==============================] - 463s 4ms/step - loss: 0.3750 - mean_absolute_error: 0.1863 - val_loss: 1.0277 - val_mean_absolute_error: 0.2752
Epoch 4/10
117001/117001 [==============================] - 455s 4ms/step - loss: 0.3851 - mean_absolute_error: 0.1852 - val_loss: 1.2904 - val_mean_absolute_error: 0.3192
Epoch 5/10
117001/117001 [==============================] - 462s 4ms/step - loss: 0.4198 - mean_absolute_error: 0.1916 - val_loss: 1.2641 - val_mean_absolute_error: 0.4141
Epoch 6/10
117001/117001 [==============================] - 461s 4ms/step - loss: 0.4226 - mean_absolute_error: 0.1918 - val_loss: 1.3638 - 

## model 5 binary target_Y ( 0 or 1 )

In [23]:
# model 5: use binary target_Y (0 or 1)
model_5 = Sequential()

model_5.add(Dense(64,
                  input_shape=(105,),
                  activation=None,
                  kernel_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                  bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None)))
# model_4.add(BatchNormalization())
model_5.add(LeakyReLU(alpha=0.2))
# model_4.add(Dropout(0.2))

model_5.add(Dense(256, activation=None))
# model_4.add(BatchNormalization())
model_5.add(LeakyReLU(alpha=0.2))
# model_4.add(Dropout(0.2))

model_5.add(Dense(128, activation=None))
# model_4.add(BatchNormalization())
model_5.add(LeakyReLU(alpha=0.2))
# model_4.add(Dropout(0.2))

model_5.add(Dense(1, activation=None))
model_5.add(Activation('sigmoid'))

# SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)?
# model_4.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),

# i tried lr=0.001 also but both didn't improve loss
# lr=0.001 > lr=0.0001
model_5.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='binary_crossentropy',
              metrics=[metrics.binary_accuracy, metrics.mae])

model_5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 64)                6784      
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 256)               16640     
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 129       
__________

In [24]:
# training model_5

# add checkpointer
save_model_name = "keiba_model_5.h5"
checkpointer = ModelCheckpoint(filepath='results/'+save_model_name, verbose=0)

minibatch_size = 64

steps_per_epoch = training_data_count // minibatch_size
validation_steps = validation_data_count // minibatch_size

model_5.fit_generator(generator=data_generator_binary(batch_size=minibatch_size, data_type='training'),
                    steps_per_epoch=steps_per_epoch,
                    validation_data=data_generator_binary(batch_size=minibatch_size, data_type='validation'),
                    validation_steps=validation_steps,
                    epochs=10,
                    callbacks=[checkpointer])

Epoch 1/10
117001/117001 [==============================] - 506s 4ms/step - loss: 0.4404 - binary_accuracy: 0.8169 - mean_absolute_error: 0.2753 - val_loss: 0.8995 - val_binary_accuracy: 0.5969 - val_mean_absolute_error: 0.4268
Epoch 2/10
117001/117001 [==============================] - 509s 4ms/step - loss: 0.4395 - binary_accuracy: 0.8200 - mean_absolute_error: 0.2715 - val_loss: 0.9431 - val_binary_accuracy: 0.6155 - val_mean_absolute_error: 0.4095
Epoch 3/10
117001/117001 [==============================] - 516s 4ms/step - loss: 0.4394 - binary_accuracy: 0.8215 - mean_absolute_error: 0.2683 - val_loss: 1.1854 - val_binary_accuracy: 0.5907 - val_mean_absolute_error: 0.4274
Epoch 4/10
117001/117001 [==============================] - 504s 4ms/step - loss: 0.4436 - binary_accuracy: 0.8168 - mean_absolute_error: 0.2720 - val_loss: 0.8432 - val_binary_accuracy: 0.6090 - val_mean_absolute_error: 0.4197
Epoch 5/10
117001/117001 [==============================] - 511s 4ms/step - loss: 0.4567

## model 6

In [8]:
# model 6: use binary target_Y (0 or 1)
model_6 = Sequential()

model_6.add(Dense(64,
                  input_shape=(105,),
                  activation=None,
                  kernel_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None),
                  bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None)))
model_6.add(BatchNormalization())
model_6.add(LeakyReLU(alpha=0.2))
# model_6.add(Dropout(0.2))

model_6.add(Dense(256, activation=None))
model_6.add(BatchNormalization())
model_6.add(LeakyReLU(alpha=0.2))
# model_6.add(Dropout(0.2))

model_6.add(Dense(128, activation=None))
model_6.add(BatchNormalization())
model_6.add(LeakyReLU(alpha=0.2))
# model_6.add(Dropout(0.2))

model_6.add(Dense(1, activation=None))
model_6.add(Activation('sigmoid'))

# SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)?
# model_5.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
model_6.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='binary_crossentropy',
              metrics=[metrics.binary_accuracy, metrics.mae])

model_6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                6784      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16640     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
__________

In [9]:
# training model_6

# add checkpointer
save_model_name = "keiba_model_6.h5"
checkpointer = ModelCheckpoint(filepath='results/'+save_model_name, verbose=0)

minibatch_size = 64

steps_per_epoch = training_data_count // minibatch_size
validation_steps = validation_data_count // minibatch_size

model_6.fit_generator(generator=data_generator_binary(batch_size=minibatch_size, data_type='training'),
                    steps_per_epoch=steps_per_epoch,
                    validation_data=data_generator_binary(batch_size=minibatch_size, data_type='validation'),
                    validation_steps=validation_steps,
                    epochs=10,
                    callbacks=[checkpointer])

Epoch 1/10
117001/117001 [==============================] - 952s 8ms/step - loss: 0.3290 - binary_accuracy: 0.8864 - mean_absolute_error: 0.1900 - val_loss: 1.0840 - val_binary_accuracy: 0.5975 - val_mean_absolute_error: 0.4167
Epoch 2/10
117001/117001 [==============================] - 937s 8ms/step - loss: 0.4297 - binary_accuracy: 0.8226 - mean_absolute_error: 0.2651 - val_loss: 0.8704 - val_binary_accuracy: 0.5967 - val_mean_absolute_error: 0.4391
Epoch 3/10
117001/117001 [==============================] - 942s 8ms/step - loss: 0.4754 - binary_accuracy: 0.7854 - mean_absolute_error: 0.3025 - val_loss: 0.7443 - val_binary_accuracy: 0.6177 - val_mean_absolute_error: 0.4233
Epoch 4/10
117001/117001 [==============================] - 945s 8ms/step - loss: 0.4994 - binary_accuracy: 0.7637 - mean_absolute_error: 0.3227 - val_loss: 0.7514 - val_binary_accuracy: 0.6072 - val_mean_absolute_error: 0.4312
Epoch 5/10
117001/117001 [==============================] - 954s 8ms/step - loss: 0.5174